In [1]:
import sys
import logging

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [131]:
from lrann.datasets import DataLoader, random_train_test_split
from lrann.estimators import ImplicitEst
from lrann.models import BilinearNet, DeepNet
from lrann.evaluation import mrr_score, precision_recall_score
from lrann.utils import is_cuda_available

In [132]:
data = DataLoader().load_movielens('100k-old')

In [133]:
train, test = random_train_test_split(data)

## Estimator using a low-rank approximation model

In [47]:
lra_model = BilinearNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
lra_est = ImplicitEst(model=lra_model, 
                      n_iter=5, 
                      use_cuda=is_cuda_available())
lra_est.fit(train, verbose=True)

Epoch 0: loss 0.27673777327537535
Epoch 1: loss 0.1704614986896515
Epoch 2: loss 0.15219719598293305
Epoch 3: loss 0.1433867601752281
Epoch 4: loss 0.13588194658756256


In [48]:
prec, recall = precision_recall_score(lra_est, test)
prec.mean(), recall.mean()

(0.10922587486744434, 0.08660114791713977)

In [49]:
mrr_score(lra_est, train).mean(), mrr_score(lra_est, test).mean()

(0.05083605810500996, 0.041096013580100975)

## Estimator using a deep neural model

In [45]:
nn_model = DeepNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
nn_est = ImplicitEst(model=nn_model, 
                     n_iter=5, 
                     use_cuda=is_cuda_available())
nn_est.fit(train, verbose=True)

Epoch 0: loss 0.23424124660491943
Epoch 1: loss 0.212724027299881
Epoch 2: loss 0.20724857063293456
Epoch 3: loss 0.20499628328084946
Epoch 4: loss 0.2040336129784584


In [9]:
prec, recall = precision_recall_score(nn_est, test)
prec.mean(), recall.mean()

(0.08833510074231177, 0.05972723965633153)

In [10]:
mrr_score(lra_est, train).mean(), mrr_score(nn_est, test).mean()

(0.05072063447153412, 0.02856018978872365)